# Data Cleaning and Variable Transformation

In [1]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/bank_customer_churn.csv')


Mounted at /content/drive


In [2]:
# Part 1 Data Acquisition

# Display the first ten rows of the data frame to examine if it is an individual-level data set
print(df.head(10))

# Display the variable list
print(df.columns.values)

print(df.shape)

   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   
5          6    15574012       Chu          645     Spain    Male   44   
6          7    15592531  Bartlett          822    France    Male   50   
7          8    15656148    Obinna          376   Germany  Female   29   
8          9    15792365        He          501    France    Male   44   
9         10    15592389        H?          684    France    Male   27   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0       2       0.00              1          1               1   
1       1   83807.86              1          0       

In [3]:
df.isnull().sum()

,0
RowNumber,0
CustomerId,0
Surname,0
CreditScore,0
Geography,0
Gender,0
Age,0
Tenure,0
Balance,0
NumOfProducts,0


In [4]:
df.drop(columns=['RowNumber','CustomerId','Surname', 'Complain'],inplace=True)

In [5]:
# Separate all the variables into two lists for future column indexing
# One for numerical, the other for categorical
cvar_list = ['Geography', 'Gender', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'Exited', 'Satisfaction Score', 'Card Type']
nvar_list = ['CreditScore', 'Age', 'Tenure', 'Balance', 'EstimatedSalary', 'Point Earned']

# Check if there is any missing value left
df.isnull().sum()

,0
CreditScore,0
Geography,0
Gender,0
Age,0
Tenure,0
Balance,0
NumOfProducts,0
HasCrCard,0
IsActiveMember,0
EstimatedSalary,0


In [6]:
# Variable Transformation

# Standardize the numerical variables
df_sample1 = df.copy()
df_sample1[nvar_list] = (df_sample1[nvar_list] - df_sample1[nvar_list].mean())/df_sample1[nvar_list].std()

# Set the datatype for the variables in the cvar_list to be categorical in Python
# Set the datatype for the variables in the nvar_list to be numerical in Python
df_sample2 = df_sample1.copy()
df_sample2[cvar_list] = df_sample1[cvar_list].astype('category')
df_sample2[nvar_list] = df_sample1[nvar_list].astype('float64')

# Convert the categorical variables into dummies (Step 1 of dummy coding)
# prefix_sep is the sympol used to create the dummy variable names.

df_sample3 = df_sample2.copy()
df_sample3 = pd.get_dummies(df_sample2, prefix_sep='_')

df_sample3
print(df_sample3.columns.values)

['CreditScore' 'Age' 'Tenure' 'Balance' 'EstimatedSalary' 'Point Earned'
 'Geography_France' 'Geography_Germany' 'Geography_Spain' 'Gender_Female'
 'Gender_Male' 'NumOfProducts_1' 'NumOfProducts_2' 'NumOfProducts_3'
 'NumOfProducts_4' 'HasCrCard_0' 'HasCrCard_1' 'IsActiveMember_0'
 'IsActiveMember_1' 'Exited_0' 'Exited_1' 'Satisfaction Score_1'
 'Satisfaction Score_2' 'Satisfaction Score_3' 'Satisfaction Score_4'
 'Satisfaction Score_5' 'Card Type_DIAMOND' 'Card Type_GOLD'
 'Card Type_PLATINUM' 'Card Type_SILVER']


In [7]:
# Identify the mode of each categorical variable and the corresponding dummy column to drop
rdummies = []
for var in cvar_list:
    mode_value = df_sample2[var].mode()[0]  # Get the mode for the variable
    dummy_to_drop = f"{var}_{mode_value}"  # Construct the dummy column name for the mode
    rdummies.append(dummy_to_drop)

# Remove the redundant dummies (Step 2 of dummy coding)
df_sample4 = df_sample3.copy()
df_sample4 = df_sample3.drop(columns=rdummies)

# Get the remaining variable list after the variable transformation
print("Remaining variables:", df_sample4.columns.values)

# Display the milestone dataframe. Compare it with the original dataframe.
print(df_sample4)
print(df)

Remaining variables: ['CreditScore' 'Age' 'Tenure' 'Balance' 'EstimatedSalary' 'Point Earned'
 'Geography_Germany' 'Geography_Spain' 'Gender_Female' 'NumOfProducts_2'
 'NumOfProducts_3' 'NumOfProducts_4' 'HasCrCard_0' 'IsActiveMember_0'
 'Exited_1' 'Satisfaction Score_1' 'Satisfaction Score_2'
 'Satisfaction Score_4' 'Satisfaction Score_5' 'Card Type_GOLD'
 'Card Type_PLATINUM' 'Card Type_SILVER']
      CreditScore       Age    Tenure   Balance  EstimatedSalary  \
0       -0.326205  0.293503 -1.041708 -1.225786         0.021885   
1       -0.440014  0.198154 -1.387468  0.117344         0.216523   
2       -1.536717  0.293503  1.032856  1.332987         0.240675   
3        0.501496  0.007456 -1.387468 -1.225786        -0.108912   
4        2.063781  0.388852 -1.041708  0.785689        -0.365258   
...           ...       ...       ...       ...              ...   
9995     1.246426  0.007456 -0.004426 -1.225786        -0.066416   
9996    -1.391870 -0.373939  1.724377 -0.306363        

# Logistic Regression

In [8]:
from sklearn.model_selection import train_test_split
# Placeholder variables: df4partition, testpart_size
# test_size specifies the percentage for the test partition
df4partition = df_sample4.copy()
testpart_size = 0.2

# random_state specifies the seed for random number generator.
# random_state = 1 unless otherwised noted
df_nontestData, df_testData = train_test_split(df4partition, test_size=testpart_size, random_state=1)

print(df_nontestData)

      CreditScore       Age    Tenure   Balance  EstimatedSalary  \
2694    -0.233089 -0.946032 -0.695947  0.587542         0.428771   
5140    -0.253781 -0.946032 -0.350186  0.469273        -1.025646   
2568    -0.398629  0.770247  0.341335  0.858032        -0.944872   
3671    -0.046856  1.246991  0.341335  0.564984        -0.551605   
7427     0.656689 -0.564637  1.032856  0.729699         1.085453   
...           ...       ...       ...       ...              ...   
2895    -0.305513  0.770247  0.687096  0.494851        -0.578868   
7813     0.346302  2.295828 -0.695947  0.076284        -0.529415   
905      0.222147  0.579549  1.378617 -1.225786        -0.140193   
5192     0.129030  0.007456  1.032856 -1.225786         0.018756   
235      1.163656  0.293503  0.341335  0.379421        -1.158524   

      Point Earned  Geography_Germany  Geography_Spain  Gender_Female  \
2694     -1.117695               True            False          False   
5140      1.024610              False

In [9]:
# Part 6 Logistic Regression with Penalty

# Required package: scikit-learn. Package name in Python: sklearn
# Required subpackage: linear_model.
# Required function name: LogisticRegression, LogisticRegressionCV

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

# Separate the predictor values and the DV values into X and y respectively
# Placeholder variable: DV
DV = 'Exited_1'
y = df_nontestData[DV]
X = df_nontestData.drop(columns=[DV])

# Run Logistic regression with pre-specified penalty level (alpha)
# Placeholder variable: alpha
alpha = 10

# The Logistic regression results are put into a model object clf
clf = LogisticRegression(C=1/alpha, penalty='l1', solver='saga', max_iter=200, random_state=1).fit(X,y)

# A user-defined function summary_coef
# to display the estimated coefficients of a model candidate obtained by the Logistic Regression analysis
def summary_coef(model_object):
  n_predictors = X.shape[1]
  model_coef = pd.DataFrame(model_object.coef_.reshape(1, n_predictors), columns=X.columns.values)
  model_coef['Intercept'] = model_object.intercept_
  return model_coef.transpose()

print(summary_coef(clf))

                             0
CreditScore          -0.044479
Age                   0.703892
Tenure               -0.024015
Balance              -0.011340
EstimatedSalary       0.010579
Point Earned         -0.007762
Geography_Germany     0.808062
Geography_Spain       0.001694
Gender_Female         0.474218
NumOfProducts_2      -1.447433
NumOfProducts_3       2.115908
NumOfProducts_4       2.113373
HasCrCard_0           0.045440
IsActiveMember_0      1.035027
Satisfaction Score_1  0.000000
Satisfaction Score_2  0.151310
Satisfaction Score_4  0.000000
Satisfaction Score_5  0.000000
Card Type_GOLD       -0.036929
Card Type_PLATINUM    0.000000
Card Type_SILVER      0.021629
Intercept            -2.182460


In [12]:
# Run Logistic regression with k-fold cross validation with k=5
# Placeholder variable: kfolds
kfolds = 5

# Here we specify within which range of the penalty levels we will search for the optimal penalty level,
# i.e., the level that leads to the best model candidate
# We search the optimal alpha within [min_alpha, max_alpha]
min_alpha = 0.001
max_alpha = 100

# We further discretize the continuous alpha range [min_alpha, max_alpha] into n individual points of alpha
# We train n model candidates each of which corresponds to one individual alpha point
n_candidates = 1000

import numpy as np

# We store the list of individual alpha points into alpha_list
alpha_list = list(np.linspace(min_alpha, max_alpha, num=n_candidates))

# C_list is the element-wise inverse of alpha_list. It is required as one of the paramater values for LogisticRegressionCV
C_list = list(1/np.linspace(min_alpha, max_alpha, num=n_candidates))

# Set n_jobs to be -1 to run LogisticRegressionCV on all CPU cores.
clf_optimal = LogisticRegressionCV(Cs=C_list, cv=kfolds, penalty='l1', solver='saga', max_iter=2000, random_state=1, n_jobs=-1).fit(X,y)

# Display the estimated coefficients of the final selected model
print(summary_coef(clf_optimal))

# Display the optimal alpha that yields the final selected model (the best model candidate)
print(1/clf_optimal.C_)
print(clf_optimal.C_)

# Calcuate the error rate over the test partition based on the final selected model

# y_test_actual is the actual values of the DV in the test partition
y_test_actual = df_testData[DV]

# X_test is the predictor values in the test partition
X_test = df_testData.drop(columns=[DV])

# Use predict method of the clf_optimal object to apply the model associated with clf_optimal to the test partition
# y_test_predicted is the predicted values of the DV in the test partition
y_test_predicted = clf_optimal.predict(X_test)

# Import the metrics package
from sklearn import metrics

# Display the confusion matrix over the test partition
print('Confusion Matrix: ')
print(metrics.confusion_matrix(y_test_actual, y_test_predicted))

# Display the accuracy over the test partition
print('Accuracy over Test Partition: ')
print(clf_optimal.score(X_test, y_test_actual))

y_test_prob = clf_optimal.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class

from sklearn.metrics import roc_auc_score

auc = roc_auc_score(y_test_actual, y_test_prob)
print(f"AUC: {auc}")

                             0
CreditScore          -0.057385
Age                   0.726884
Tenure               -0.038660
Balance              -0.036514
EstimatedSalary       0.018547
Point Earned         -0.017508
Geography_Germany     0.920304
Geography_Spain       0.102293
Gender_Female         0.514588
NumOfProducts_2      -1.498060
NumOfProducts_3       2.455100
NumOfProducts_4       6.388572
HasCrCard_0           0.093079
IsActiveMember_0      1.112184
Satisfaction Score_1 -0.007065
Satisfaction Score_2  0.203601
Satisfaction Score_4  0.014536
Satisfaction Score_5 -0.028886
Card Type_GOLD       -0.105997
Card Type_PLATINUM   -0.058679
Card Type_SILVER      0.041304
Intercept            -2.314957
[0.2011982]
[4.97022344]
Confusion Matrix: 
[[1528   57]
 [ 259  156]]
Accuracy over Test Partition: 
0.842
AUC: 0.8483280757097791
